In [1]:
# Imports
# [] Need to install mahotas library for morphological operations
import os 
import numpy as np
import mahotas as mh
import matplotlib.pylab as plt
import glob
from itertools import zip_longest
from PST_function import PST

In [2]:
#Generating initial Population
import random

def generatePopulation(populationNumber):
    randomPopulation = np.zeros((populationNumber, 4))
    
    for i in range (populationNumber):
        LPF = random.uniform(.05,.12) # Gaussian Low Pass Filter
        #LPF = .1
        Phase_strength = random.uniform(.3,.7)
        Warp_strength = random.uniform(5,50)
        Color_filter = random.uniform(.0001, .0009)
        #Color_filter = .0005
        
        randomPopulation[i,:] = [LPF, Phase_strength, Warp_strength, Color_filter]
        
    return randomPopulation

In [67]:
# Find the percent of pixels on the annotated vessel that are left uncolored: 
def falseNegfalsePos(Edge, Image_annotated):     
    white_count = np.count_nonzero(Image_annotated)
    error = np.subtract(Edge, Image_annotated)
    falseNeg = np.count_nonzero(error==(-1))
    falseNeg = float(falseNeg)/float(white_count) 
    falsePos = np.count_nonzero(error==1) 
    falsePos = float(falsePos)/float(Edge.size-white_count) 

    return(falseNeg, falsePos)

# Evaluate error over the dataset 
# Threshold values of the original image so that the black part doesn't affect things 
# THRESHOLD PARAMETERS CAN BE LEARNED TOO 



# creates a list of edge images given a set of parameters 
def create_edge_list(original_image_list, annotated_image_list, parameter_list): # takes 1D parameter array 
    edge_list = [] 
    for i in range(len(annotated_image_list)):
        [Edge, PST_Kernel]= PST(original_image_list[i], parameter_list[0], parameter_list[1], parameter_list[2], -1, 1, 0)
        # apply threshold to pick greys and make them white 
        Edge[Edge > parameter_list[3]] = 1   
        Edge[Edge < parameter_list[3]] = 0 
        # apply threshold to get rid of the circle around the image 
        Edge[(original_image_list[i] < 50) + (original_image_list[i] > 200)] = 0 
        edge_list.append(Edge) 
    return edge_list 

# average of all the percent errors over a list of images
def all_images_error(edge_list, annotated_image_list): 
    errors = [] 
    for i in range(len(annotated_image_list)): 
        error = (0.1*falseNegfalsePos(edge_list[i], annotated_image_list[i])[0]) + (0.9*falseNegfalsePos(edge_list[i], annotated_image_list[i])[1])
        errors.append(error) # false positives are weighted higher 
    #print("Here is a list of errors for a set of images: ")
    #print(errors)
    #print("Average of this set: ")
    #print(np.mean(errors))
    return np.mean(errors) 
        
# creates list of average errors (1D) given a list of parameter sets (2D) 
# is this compute_fitness ?? 
def computeFitness(parameter_matrix, original_image_list, annotated_image_list):
    error_list = [] 
    for j in range(parameter_matrix.shape[0]): # over each set of parameters, or number of rows of parameter matrix
        # for each set of parameters, create a list of edge images 
        #print("Working on parameter set " + str(j) + " with the parameters: ") 
        #print(parameter_matrix[j,:])
        edge_list = create_edge_list(original_image_list, annotated_image_list, parameter_matrix[j,:]) 
        error_list.append(all_images_error(edge_list, annotated_image_list))  
        #print("Just added " + str(error_list[-1]) + " to the list") 
    fitnessList = []
    for i in range(len(error_list)): 
        fitnessList.append(1-error_list[i]) 
    return fitnessList

In [4]:
#selection takes in a list of average error of parameter list performed on every image
#returns the index in original population array of fittest, second fittest, and leastfit

def selection(calculatedFitness):
    sortederror = np.sort(calculatedFitness)
    fittest = sortederror[-1]
    secondfittest = sortederror[-2]
    
    #check to make sure program doesnt pick a child that is exactly the same as parent
    index = -3
    while secondfittest == fittest:
        if (index == -1*len(calculatedFitness)):
            break
        secondfittest = sortederror[index]
        index = index - 1
    
    for i in range(len(calculatedFitness)):
        if fittest == calculatedFitness[i]:
            fittestidx = i
        if secondfittest == calculatedFitness[i]:
            secondfittestidx = i
    
    return fittestidx, secondfittestidx

In [5]:
#crossover takes in parameters for fittest, 2nd fittest, and total population number
#returns an array of populationNumber - 4 new children with traits based on the parents

def crossover(fittest, secondfittest, populationNumber):
    newchildren = np.zeros((populationNumber-4,4))
    for k in range (populationNumber-4):
        crossoverpoint = random.randint(0,4)
        for i in range (crossoverpoint):
            newchildren[k,i] = fittest[i]
        for i in range (4-crossoverpoint):
            newchildren[k,crossoverpoint+i] = secondfittest[crossoverpoint+i]
    
    return newchildren

In [17]:
#mutation takes in list of children and mutates them
#returns list of mutated children

def mutation(newchildren, fittest, secondfittest):
    for i in range(newchildren.shape[0]):
        for j in range(newchildren.shape[1]):
            mutate = 0;
            mutationProbability = random.randint(1,100)
            if (mutationProbability < 20):
                mutate = 1
            if mutate == 1: #mutate at most by 10% of total range of values
                if j == 0: #mutate lPF  
                    mutateby = random.uniform(-.007,.007)
                    #mutateby = 0
                    while (newchildren[i,j] + mutateby >.12) or (newchildren[i,j] + mutateby < .05):
                        mutateby = random.uniform(-.007,.007)
                elif j == 1: #mutate Phase_Strength
                    mutateby = random.uniform(-.04,.04)
                    while (newchildren[i,j] + mutateby >.7) or (newchildren[i,j] + mutateby < .3):
                        mutateby = random.uniform(-.04,.04)
                elif j == 2: #mutate warp Strength
                    mutateby = random.uniform(-4.5,4.5)
                    while (newchildren[i,j] + mutateby >50) or (newchildren[i,j] + mutateby < 5):
                        mutateby = random.uniform(-4.5,4.5)
                else:         #mutate color threshold
                    #mutateby = 0
                    mutateby = random.uniform(-.00008,.00008)
                    while (newchildren[i,j] + mutateby >.0009) or (newchildren[i,j] + mutateby < .0001):
                        mutateby = random.uniform(-.00008,.00008)
                newchildren[i,j] += mutateby
        if np.array_equal(newchildren[i,:],fittest) or np.array_equal(newchildren[i,:],secondfittest):
            i = i - 1
            
    return newchildren

In [7]:
#create newpopulation takes in newchildrenlist, parents
#returns a newpopulation list with newchildren, parents, and two randomly generated parents

def createNewPopulation(fittest, secondfittest, newchildren):
    newPopulation = np.zeros((len(newchildren)+4, 4))
    newPopulation[0,:] = fittest
    newPopulation[1,:] = secondfittest
    for i in range (newchildren.shape[0]):
        newPopulation[i+2] = newchildren[i,:]
    randGenerated = generatePopulation(2)
    newPopulation[-1,:] = randGenerated[0,:]
    newPopulation[-2,:] = randGenerated[1,:]
    
    return newPopulation

In [68]:
#convergecheck checks if the difference between difference in fitness of new children is negligible from parents 
#returns 1 if convergence is reached
# This version of the function was not used 
'''
def convergeCheck (calculatedFitness):
    avgFitness = 0;
    for i in range (len(calculatedFitness) - 4):
        avgFitness += calculatedFitness[2 + i]
    avgFitness = avgFitness / (len(calculatedFitness) - 4)
    
    #compare to parents
    comparetoFittest = abs(calculatedFitness[0] - avgFitness)
    comparetoSecondFittest = abs(calculatedFitness[1] - avgFitness)
    
    if (abs(comparetoFittest - comparetoSecondFittest) < .0001 ): #.05 is what we defined as negligible difference
        return 1
    
    return 0
'''

def convergeCheck (calculatedFitness):
    sortedFitness = np.sort(calculatedFitness)
    fittest = sortedFitness[-1]
    secondfittest = sortedFitness[-2]
    
    #check to make sure program doesnt pick a child that is exactly the same as parent
    index = -3
    while secondfittest == fittest:
        if (index == -1*len(calculatedFitness)):
            break
        secondfittest = sortedFitness[index]
        index = index - 1
    
    difference = abs(fittest - secondfittest)
    print ("difference between fittest and secondfittest: " + str(difference))
    if (fittest > .9) and (difference < .000000001):
        return 1
    
    return 0

In [32]:
# This is the subset of the original images which are annotated
# pass in the original image list and it picks out the ones we have the annotated versions of 
def create_original_images_subset(original_image_list): 
    original_image_list_train = [] 
    original_image_list_train.append(original_image_list[0]) #0
    original_image_list_train.append(original_image_list[1]) #1
    original_image_list_train.append(original_image_list[2]) #2
    original_image_list_train.append(original_image_list[3]) #3
    original_image_list_train.append(original_image_list[4]) #4 
    original_image_list_train.append(original_image_list[43])#5 
    original_image_list_train.append(original_image_list[75])#6 
    original_image_list_train.append(original_image_list[79])#7
    original_image_list_train.append(original_image_list[80])#8
    original_image_list_train.append(original_image_list[135])#9
    original_image_list_train.append(original_image_list[157])#10
    original_image_list_train.append(original_image_list[158])#11
    original_image_list_train.append(original_image_list[229])#12
    original_image_list_train.append(original_image_list[230])#13
    original_image_list_train.append(original_image_list[233])#14
    original_image_list_train.append(original_image_list[234])#15
    original_image_list_train.append(original_image_list[249])#16
    original_image_list_train.append(original_image_list[285])#17
    original_image_list_train.append(original_image_list[313])#18
    original_image_list_train.append(original_image_list[318])#19
    return original_image_list_train 

In [10]:
# Create the list of annotated images 
def import_annotated_images(filepath): 
    annotated_image_list = []
    for filename in glob.glob(filepath): 
        im=mh.imread(filename)
        im[im == 255] = 1
        annotated_image_list.append(im)
    return annotated_image_list

In [11]:
def import_original_images(filepath):
    # Create original image list 
    original_image_list = [] 
    for filename in glob.glob(filepath): 
        im=mh.imread(filename)
        original_image_list.append(mh.colors.rgb2grey(im))
    return original_image_list 

In [34]:
#import images
annotated_path = '../labels-ah/*.ppm'
annotated_list = import_annotated_images(annotated_path)
original_images_filepath = '../all-images/*.ppm'
original_images_list = import_original_images(original_images_filepath) 
original_images_subset = create_original_images_subset(original_images_list)

In [69]:
#Main function

#populate
populationNumber = 10;
population = generatePopulation(populationNumber)
computedFitness = computeFitness(population, original_images_subset, annotated_list)
print("0 Generation Fitness")
print (computedFitness)

#run genetic algorithm
fitnesses = [] # keep a list of the most fit's fitness 
terminate = 0
generationCount = 0;
while (terminate == 0):
    [fittestidx, secondfittestidx] = selection(computedFitness)
    fittest = population[fittestidx,:]
    secondfittest = population[secondfittestidx,:]
    print ("Fittest for Generation " + str(generationCount) + " : " + str(fittest))
    print ("SecondFittest for Generation " + str(generationCount) + " : " + str(secondfittest))
    newchildren = crossover(fittest,secondfittest,populationNumber)
    mutatedchildren = mutation(newchildren, fittest, secondfittest)
    population = createNewPopulation(fittest, secondfittest, newchildren)
    computedFitness = computeFitness(population, original_images_subset, annotated_list)
    fitnesses.append(computedFitness[0]) # add to the list of the fitnesses 
    print (str(generationCount + 1) + " Generation Fitness")
    print (computedFitness)
    terminate = convergeCheck(computedFitness)
    generationCount = generationCount + 1
    
print ("fittest:")
print (fittest)
print ("second fittest:")
print (secondfittest)

0 Generation Fitness
[0.9443919526000033, 0.9098337666722881, 0.9301878508332909, 0.9441509283339122, 0.9027373403927687, 0.8783677264633012, 0.9365972520920149, 0.9287024353335307, 0.9299907229257566, 0.9297586554294657]
Fittest for Generation 0 : [1.12033937e-01 4.88420770e-01 4.39718528e+01 7.83485759e-04]
SecondFittest for Generation 0 : [8.98775024e-02 5.30739937e-01 2.66633310e+01 5.14324259e-04]
1 Generation Fitness
[0.9443919526000033, 0.9441509283339122, 0.9392590193941932, 0.9438275577428974, 0.9365108903753557, 0.9401639323358246, 0.9412483647641549, 0.9441509283339122, 0.9432609765350938, 0.9437749036091473]
difference between fittest and secondfittest: 0.0002410242660910722
Fittest for Generation 1 : [1.12033937e-01 4.88420770e-01 4.39718528e+01 7.83485759e-04]
SecondFittest for Generation 1 : [8.98775024e-02 5.30739937e-01 2.66633310e+01 5.14324259e-04]
2 Generation Fitness
[0.9443919526000033, 0.9441509283339122, 0.9387818463030905, 0.9441509283339122, 0.944245325976177,

18 Generation Fitness
[0.9445399158001984, 0.9445352658127051, 0.9445333066542754, 0.9445352658127051, 0.9445437560976042, 0.9445352658127051, 0.9445352658127051, 0.9443461866435664, 0.9332165816180876, 0.890208058535028]
difference between fittest and secondfittest: 3.840297405854898e-06
Fittest for Generation 18 : [1.05234372e-01 5.01224578e-01 4.94550607e+01 7.74150626e-04]
SecondFittest for Generation 18 : [1.05234372e-01 5.01224578e-01 4.94550607e+01 7.83485759e-04]
19 Generation Fitness
[0.9445437560976042, 0.9445399158001984, 0.9445399158001984, 0.9445171325110873, 0.944471175578564, 0.944409995455679, 0.9445465868603676, 0.9445437560976042, 0.8999617158917813, 0.9419861965448092]
difference between fittest and secondfittest: 2.830762763417205e-06
Fittest for Generation 19 : [1.05234372e-01 5.01224578e-01 4.86592549e+01 7.74150626e-04]
SecondFittest for Generation 19 : [1.05234372e-01 5.01224578e-01 4.94550607e+01 7.74150626e-04]
20 Generation Fitness
[0.9445465868603676, 0.9445

KeyboardInterrupt: 

In [1]:
# Graphing convergence of fitness function 

#print(fitnesses)
generation_list = range(0,20) # set to # generations in fitnesses
fig = plt.figure()
plt.ylim(0.9442,0.9446)
plt.xlabel("Generation")
plt.ylabel("Fitness Score") 
plt.title("Fitness Convergence for (0.1, 0.9)")
plt.scatter(generation_list, fitnesses[0:20]); #set to # generations in fitnesses

NameError: name 'fitnesses' is not defined

In [1]:
# Used to look at the results for different parameter sets, copied from the print statement of the genetic algorithm script 
#Test for weighted 30 white 70 black 
test_image = original_images_subset[16] # 16 is the pic (out of 20) chosen to display 
test_annotation =annotated_list[16]
gen_0_params = [1.19324527e-01, 3.28020122e-01, 4.98855156e+01, 3.90786859e-04]
gen_1_params = [1.19324527e-01, 3.28020122e-01, 4.98855156e+01, 3.90786859e-04]
gen_10_params = [1.05234372e-01, 5.01224578e-01, 4.86592549e+01, 7.74150626e-04]
gen_25_params = [8.55495315e-02, 6.56790455e-01, 3.94842330e+01, 3.65756424e-04]
gen_50_params = [1.19324527e-01, 3.28020122e-01, 4.98855156e+01, 3.90786859e-04]


[Edge, PST_Kernel]= PST(test_image, gen_10_params[0], gen_10_params[1], gen_10_params[2], -1,1,0)
#print(Edge[0,:])
# if the pixel is greater than gen_10_params[3] make it white 
Edge[Edge > gen_10_params[3]] = 1
# if the pixel is greater than gen_10_params[3] make it black
Edge[Edge < gen_10_params[3]] = 0 
# more thresholding to eliminate the black border or white spots 
Edge[(test_image < 50) + (test_image > 200)] = 0 
    
# experimenting with different filters to eliminate speckle noise
Edge_filter = median_filter(Edge, footprint=np.ones((5,5)))



                       
def imshow_pair(image_pair, titles=('', ''), figsize=(10, 6), **kwargs):
    fig, axes = plt.subplots(ncols=len(image_pair), figsize=figsize)
    for ax, img, label in zip_longest(axes.ravel(), image_pair, titles, fillvalue=''):
        ax.imshow(img, **kwargs)
        ax.set_title(label)
# show the original image, detected edges and an overlay of the original image with detected edges       
print('      Original Image        Professional Annotation    Edge Detected using PST')
imshow_pair((test_image, test_annotation, Edge), cmap='gray')
    

NameError: name 'PST' is not defined